<a href="https://colab.research.google.com/github/VG1234/NLP_PROJECT/blob/main/NLP_TEAM5_copy4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Natural Language Processing Course Project


> Naimisha Pinna (*npinn001@odu.edu*)

> Venugopal Nagandla (*vnaga002@odu.edu*)








In [ ]:
# For sending GET requests from the API
import requests
# For saving access tokens and for file management when creating and adding to the dataset
import os
# For dealing with json responses we receive from the API
import json
# For displaying the data after
import pandas as pd
# For saving the response data in CSV format
import csv
# For parsing the dates received from twitter in readable formats
import datetime
import dateutil.parser
import unicodedata
#To add wait time between requests
import time
#Installing selenium to web scrap the information from the url link
!apt update
# !apt install chromium-chromedriver
# !pip install selenium
# !pip install transformers


Hit:1 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:4 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:6 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:7 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:8 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Ign:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Hit:11 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:12 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:13 https://developer.downlo

## Twitter API 

In [ ]:
os.environ['TOKEN'] = 'AAAAAAAAAAAAAAAAAAAAAAq9jQEAAAAAwnZ%2BRfR5T7y15H%2FCcJSkIy0q2Ps%3DCmMg1XbTxKq6oS03TqVfhdLKNmPg9QLtGFJ9hFgtqAydKZWUcO';

In [ ]:
def auth():
    return os.getenv('TOKEN')

In [ ]:
def create_headers(bearer_token):
    headers = {"Authorization": "Bearer {}".format(bearer_token)}
    return headers

In [ ]:
def create_url( accountid,max_results = 10):
    
    search_url = "https://api.twitter.com/2/users/"+accountid+"/tweets" #Change to the endpoint you want to collect data from

    #change params based on the endpoint you are using
    query_params = {
                    'max_results': max_results,
                    'tweet.fields': 'id,text,author_id,lang,entities',
                    'next_token': {}}
    return (search_url, query_params)

In [ ]:
def connect_to_endpoint(url, headers, params, next_token = None):
    params['pagination_token'] = next_token   #params object received from create_url function
    response = requests.request("GET", url, headers = headers, params = params)
    print("Endpoint Response Code: " + str(response.status_code))
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    return response.json()

In [ ]:
#Inputs for the request
bearer_token = auth()
headers = create_headers(bearer_token)
max_results = 50

In [ ]:
def append_to_csv(json_response, fileName):

    #A counter variable
    counter = 0

    #Open OR create the target CSV file
    csvFile = open(fileName, "a", newline="", encoding='utf-8')
    csvWriter = csv.writer(csvFile)

    #Loop through each tweet
    for tweet in json_response['data']:
        
        # We will create a variable for each since some of the keys might not exist for some tweets
        # So we will account for that

        # 1. Author ID
        author_id = tweet['author_id']

       
        # 4. Tweet ID
        tweet_id = tweet['id']

        # 5. Language
        lang = tweet['lang']

        
        # 8. Tweet text
        text = tweet['text']
        if 'urls' in tweet['entities']:
          url=tweet['entities'].get('urls')[0].get('url')
        else: 
          url=''
        print(url);
        # Assemble all data in a list
        res = [id,text,author_id,lang,url]
        
        # Append the result to the CSV file
        csvWriter.writerow(res)
        counter += 1
    csvFile.close()


In [ ]:
#Inputs for tweets
bearer_token = auth()
headers = create_headers(bearer_token)

max_results = 99
#Total number of tweets we collected from the loop
total_tweets = 0

# Create file
csvFile = open("twitterdata.csv", "a", newline="", encoding='utf-8')
csvWriter = csv.writer(csvFile)

#Create headers for the data you want to save, in this example, we only want save these columns in our dataset
csvWriter.writerow(['id','text','authorid','lang','url'])
csvFile.close()
accounts=['1520115242134642690','4069075997','587714983'];
for i in range(0,3):
  if total_tweets>3000:
    break;
  # Inputs
  count = 0 # Counting tweets per time period
  max_count = 1000 # Max tweets per time period
  flag = True
  next_token = None

  # Check if flag is true
  while flag:
    # Check if max_count reached
    if count >= max_count:
        break
    print("-------------------")
    print("Token: ", next_token)
    url = create_url( accounts[i],max_results)
    print(url)
    json_response = connect_to_endpoint(url[0], headers, url[1], next_token)
    result_count = json_response['meta']['result_count']

    if 'next_token' in json_response['meta']:
        # Save the token to use for next call
        next_token = json_response['meta']['next_token']
        print("Next Token: ", next_token)
        print(result_count)
        print(next_token)
        if result_count is not None and result_count > 0 and next_token is not None:
            #print("Start Date: ", start_list[i])
            print('I am here in if')
            append_to_csv(json_response, "twitterdata.csv")
            count += result_count
            total_tweets += result_count
            print("Total # of Tweets added: ", total_tweets)
            print("-------------------")
            time.sleep(5)                
    # If no next token exists
    else:
        if result_count is not None and result_count > 0:
            print("-------------------")
            #print("Start Date: ", start_list[i])
            append_to_csv(json_response, "twitterdata.csv")
            count += result_count
            total_tweets += result_count
            print("Total # of Tweets added: ", total_tweets)
            print("-------------------")
            time.sleep(5)
        
        #Since this is the final request, turn flag to false to move to the next time period.
        flag = False
        next_token = None
    time.sleep(5)
print("Total number of results: ", total_tweets)

-------------------
Token:  None
('https://api.twitter.com/2/users/1520115242134642690/tweets', {'max_results': 99, 'tweet.fields': 'id,text,author_id,lang,entities', 'next_token': {}})
Endpoint Response Code: 200
Next Token:  7140dibdnow9c7btw424c4d77b06ajlw498o74n929gl1
99
7140dibdnow9c7btw424c4d77b06ajlw498o74n929gl1
I am here in if
https://t.co/eDRMYUw8jY
https://t.co/g5TFJw9ACs
https://t.co/f0zL3IBgnt
https://t.co/VeHrO9mFQc
https://t.co/onSQeRkCRQ
https://t.co/qBsmkBIjpx

https://t.co/6rg81LEQcG

https://t.co/z602XrabPn
https://t.co/nFZCEvASSF

https://t.co/DIuDfjuIJq
https://t.co/t9IOAcLrLv

https://t.co/IH82n2eNK2
https://t.co/PCmQWfy0mo
https://t.co/TEXJgUitmZ

https://t.co/MvneZCrMAe
https://t.co/ZpQFPCTv9Q
https://t.co/argJf4U2XC
https://t.co/heNtPL2Asz

https://t.co/K2LikwAHaz
https://t.co/XB7IKiQL4u

https://t.co/cFvsAsArYx
https://t.co/eFnwf8HkQk

https://t.co/H8SVETS50N

https://t.co/dJ7AVSTVKE
https://t.co/vvL73TrSvH
https://t.co/1bEuHMThfh
https://t.co/sroQjPr5Fb
https

## Naive Bayes Classifier

> This block of code is used to identify whether a tweet is a question or not from the above dataset



In [ ]:
import nltk.corpus
#from nltk.corpus import nps_chat
nltk.download('nps_chat')
nltk.download('punkt')
from nltk.tokenize import TweetTokenizer
 
class QuestionDetector():
 
    #Class Initialier:
    #- Creates naive bayes classifier using nltk nps_chat corpus.
    #- Initializes Tweet tokenizer
    #- Initializes question words set to be used
    def __init__(self):
        posts = nltk.corpus.nps_chat.xml_posts()
        featuresets = [(self.__dialogue_act_features(post.text), post.get('class')) for post in posts]
        print(featuresets)
        size = int(len(featuresets) * 0.1)
        train_set, test_set = featuresets[size:], featuresets[:size]
        self.classifier = nltk.NaiveBayesClassifier.train(train_set)
        Question_Words = ['what', 'where', 'when','how','why','did','do','does','have','has','am','is','are','can','could','may','would','will','should'
"didn't","doesn't","haven't","isn't","aren't","can't","couldn't","wouldn't","won't","shouldn't",'?']
        self.Question_Words_Set = set(Question_Words)
        self.tknzr = TweetTokenizer()
    #Private method, Gets the word vector from sentance
    def __dialogue_act_features(self,sentence):
         features = {}
         for word in nltk.word_tokenize(sentence):
             features['contains({})'.format(word.lower())] = True
         return features
    #Public Method, Returns 'True' if sentance is predicted to be a question, returns 'False' otherwise
    def IsQuestion(self,sentence):
        if "?" in sentence:
            return True
        tokens = self.tknzr.tokenize(sentence.lower())
        if self.Question_Words_Set.intersection(tokens) == False:
            return False
        predicted = self.classifier.classify(self.__dialogue_act_features(sentence))
        if predicted == 'whQuestion' or predicted == 'ynQuestion':
            return True
         
        return False
    

[nltk_data] Downloading package nps_chat to /root/nltk_data...
[nltk_data]   Package nps_chat is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
# import pandas module 
import pandas as pd 
import numpy as np
# making dataframe 
df = pd.read_csv("DatasetforNLP.csv") 
# output the dataframe
#print(df)
'''tx=QuestionDetector();
print(tx.IsQuestion('How are you doing'))'''
#foo = lambda x = Foo(): (x.parse("All set"), x)[-1]

#Add new column as question to check if the text of a tweet is a question
df['question'] = df['text'].apply(lambda t,x= QuestionDetector(): x.IsQuestion(t))
#print(df)

#Filter the dataframe to get only those rows where the text in tweet is a question
#dfquestion = df[df['question'] == True] 

#print(dfquestion)
df.drop('id',inplace=True,axis=1)
df.drop('authorid',inplace=True,axis=1)
df.drop('lang',inplace=True,axis=1)
df.drop('url',inplace=True,axis=1)
df.rename(columns = {'question':'label'}, inplace = True)
columns_titles = ["label","text"]
df=df.reindex(columns=columns_titles);
df['label'] = df['label'].replace([False],0)
df['label'] = df['label'].replace([True],1)
print(df);
df.to_csv('DatasetforClassification.csv');



NameError: ignored

## Classification

In [ ]:
df = pd.read_csv('DatasetforNLP.csv')
df.head()
df.describe()
df = df.dropna()

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(df['text'], df['label'], test_size = 0.3, random_state = 0)

In [ ]:
print('rows in test set: ' + str(x_test.shape))
print('rows in train set: ' + str(x_train.shape))

rows in test set: (300,)
rows in train set: (699,)




> TF-IDF Vectorizer



In [ ]:
lst = x_train.tolist()
vectorizer = TfidfVectorizer(
input= lst ,  # input is the actual text
lowercase=True,      # convert to lowercase before tokenizing
# stop_words='english' # remove stop words
)
features_train_transformed = vectorizer.fit_transform(lst) #gives tf idf vector for x_train
features_test_transformed  = vectorizer.transform(x_test) #gives tf idf vector for x_test

In [ ]:
print(features_test_transformed)

  (0, 3401)	0.12674966616548952
  (0, 3369)	0.2977176989613575
  (0, 3365)	0.3893189377323745
  (0, 3117)	0.11783579820780256
  (0, 3087)	0.16105845153958742
  (0, 2975)	0.2977176989613575
  (0, 2653)	0.06175737218332851
  (0, 2281)	0.3164259107847221
  (0, 2191)	0.0997553529063025
  (0, 2158)	0.18570108558657664
  (0, 2103)	0.3164259107847221
  (0, 1658)	0.11752508910372761
  (0, 1496)	0.27414813950208305
  (0, 839)	0.10325580832712579
  (0, 682)	0.3164259107847221
  (0, 453)	0.3164259107847221
  (0, 195)	0.2586232675722605
  (1, 3404)	0.1738182743450089
  (1, 3104)	0.31645040991757106
  (1, 3080)	0.233771998545156
  (1, 2938)	0.2844660343804601
  (1, 2932)	0.31645040991757106
  (1, 2925)	0.26575637408226704
  (1, 2846)	0.2844660343804601
  (1, 2712)	0.1782162386863029
  :	:
  (298, 1150)	0.09342816006742315
  (298, 952)	0.2045039436942696
  (298, 945)	0.2672145655096243
  (298, 815)	0.15798359033438483
  (298, 780)	0.1887052795411197
  (298, 692)	0.0867601460125692
  (298, 467)	0.087

## Naive Bayes Multinominal Classifier

In [ ]:
from sklearn.naive_bayes import MultinomialNB
# train the model
classifier = MultinomialNB()
classifier.fit(features_train_transformed, y_train)

MultinomialNB()

In [ ]:
print("classifier accuracy {:.2f}%".format(classifier.score(features_test_transformed, y_test) * 100))

classifier accuracy 71.00%


In [ ]:
labels = classifier.predict(features_test_transformed)
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
actual = y_test.tolist()
predicted = labels
results = confusion_matrix(actual, predicted)
print('Confusion Matrix :')
print(results)
print ('Accuracy Score :',accuracy_score(actual, predicted))
print ('Report : ')
print (classification_report(actual, predicted) )
score_2 = f1_score(actual, predicted, average = 'binary')
print('F-Measure: %.3f' % score_2)

Confusion Matrix :
[[117  39]
 [ 48  96]]
Accuracy Score : 0.71
Report : 
              precision    recall  f1-score   support

         0.0       0.71      0.75      0.73       156
         1.0       0.71      0.67      0.69       144

    accuracy                           0.71       300
   macro avg       0.71      0.71      0.71       300
weighted avg       0.71      0.71      0.71       300

F-Measure: 0.688


## DecisionTree Classifier

In [ ]:
from sklearn.tree import DecisionTreeClassifier
# from sklearn.pipeline import Pipeline
# from sklearn.feature_extraction.text import CountVectorizer
# from sklearn.feature_extraction.text import TfidfTransformer
# pipe = Pipeline([('vect', CountVectorizer()),
#                  ('tfidf', TfidfTransformer()),
#                  ('model', DecisionTreeClassifier(criterion= 'entropy',
#                                            max_depth = 20, 
#                                            splitter='best', 
#                                            random_state=42))])
classifier = DecisionTreeClassifier()
classifier.fit(features_train_transformed, y_train)
print("classifier accuracy {:.2f}%".format(classifier.score(features_test_transformed, y_test) * 100))

classifier accuracy 73.67%


In [ ]:
labels = classifier.predict(features_test_transformed)
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
actual = y_test.tolist()
predicted = labels
results = confusion_matrix(actual, predicted)
print('Confusion Matrix :')
print(results)
print ('Accuracy Score :',accuracy_score(actual, predicted))
print ('Report : ')
print (classification_report(actual, predicted) )
score_2 = f1_score(actual, predicted, average = 'binary')
print('F-Measure: %.3f' % score_2)

Confusion Matrix :
[[109  47]
 [ 32 112]]
Accuracy Score : 0.7366666666666667
Report : 
              precision    recall  f1-score   support

         0.0       0.77      0.70      0.73       156
         1.0       0.70      0.78      0.74       144

    accuracy                           0.74       300
   macro avg       0.74      0.74      0.74       300
weighted avg       0.74      0.74      0.74       300

F-Measure: 0.739


##Stochastic Gradient Descent Classifier

In [ ]:
from sklearn.linear_model import SGDClassifier
classifier = SGDClassifier()
classifier.fit(features_train_transformed, y_train)
print("classifier accuracy {:.2f}%".format(classifier.score(features_test_transformed, y_test) * 100))

classifier accuracy 74.00%


In [ ]:
labels = classifier.predict(features_test_transformed)

actual = y_test.tolist()
predicted = labels
results = confusion_matrix(actual, predicted)
print('Confusion Matrix :')
print(results)
print ('Accuracy Score :',accuracy_score(actual, predicted))
print ('Report : ')
print (classification_report(actual, predicted) )
score_2 = f1_score(actual, predicted, average = 'binary')
print('F-Measure: %.3f' % score_2)

Confusion Matrix :
[[119  37]
 [ 41 103]]
Accuracy Score : 0.74
Report : 
              precision    recall  f1-score   support

         0.0       0.74      0.76      0.75       156
         1.0       0.74      0.72      0.73       144

    accuracy                           0.74       300
   macro avg       0.74      0.74      0.74       300
weighted avg       0.74      0.74      0.74       300

F-Measure: 0.725


## Extreme Gradient Boost Classifier

In [ ]:
from xgboost import XGBClassifier
classifier = XGBClassifier()
classifier.fit(features_train_transformed, y_train)
print("classifier accuracy {:.2f}%".format(classifier.score(features_test_transformed, y_test) * 100))

classifier accuracy 80.33%


In [ ]:
labels = classifier.predict(features_test_transformed)

actual = y_test.tolist()
predicted = labels
results = confusion_matrix(actual, predicted)
print('Confusion Matrix :')
print(results)
print ('Accuracy Score :',accuracy_score(actual, predicted))
print ('Report : ')
print (classification_report(actual, predicted) )
score_2 = f1_score(actual, predicted, average = 'binary')
print('F-Measure: %.3f' % score_2)

Confusion Matrix :
[[134  22]
 [ 37 107]]
Accuracy Score : 0.8033333333333333
Report : 
              precision    recall  f1-score   support

         0.0       0.78      0.86      0.82       156
         1.0       0.83      0.74      0.78       144

    accuracy                           0.80       300
   macro avg       0.81      0.80      0.80       300
weighted avg       0.81      0.80      0.80       300

F-Measure: 0.784
